## Healthcare Data Q&A Assistant with RAG

### Executive Summary

This proof of concept demonstrates how structured healthcare data can be transformed into a natural-language Q&A assistant using a retrieval-augmented generation (RAG) pipeline. The system allows users to ask questions such as "What was the heart disease mortality rate in Texas in 2019?" and receive answers grounded in real data. The pipeline combines sentences embeddings, vector search with FAISS, a simple query classification layer, and a large language model (LLM) via Ollama. While the prototype is not production-ready, it validates the feasibility of the approach and highlights both strengths (accurate  lookup queries) and limitations (aggregation queries and hallucinations).

### Introduction

Healthcare data is often stored in large tabular datasets, making it difficult for non-technical users to extract insights. Analysts typically rely on SQL queries or dashboards, which require technical expertise. Natural language interfaces offer a more intuitive alternative, but raw LLMs are prone to hallucinations and cannot directly "understand" structured data.

This project explores the use of retrieval-augmented generation (RAG) as a bridge between structured healthcare data and natural-language questions. The central idea is to represent each data row as a natural-language "fact", retrieve the most relevant facts for a given query, and let the LLM generate an answer constrained by those facts.

In [1]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
import faiss
import subprocess
import json
import warnings
import os
warnings.filterwarnings("ignore")
os.environ["TOKENIZERS_PARALLELISM"] = "false"

/Users/llingcaro/Documents/Work/git_repos/healthcare_data_explorer/venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Data Source

The dataset used in this project was obtained from the CDC WONDER database (Centers for Disease Control and Prevention). This project uses the data solely for educational and research purposes.

### Dataset Description

The dataset used in this POC contains mortality rates by US state, cause of death, and year. For example, one row might state that the heart disease mortality rate in Texas in 2019 was 150. The dataset was cleaned and formatted into a structured CSV. A quick inspection shows rows with columns for state, year, cause of death, number of deaths, population, and crude mortality rate.

In [2]:
df = pd.read_csv("../poc/data/Underlying Cause of Death, 1999-2020.xls", sep="\t")
df = df.rename(
    columns = {
        'State':'state',
        'Year':'year',
        'ICD-10 113 Cause List':'cause',
        'Deaths':'deaths',
        'Population':'population',
        'Crude Rate':'crude_rate'
    }
)
df = df[['state','year','cause','deaths','population','crude_rate']]
df['crude_rate'] = pd.to_numeric(df['crude_rate'], errors='coerce')
df['year'] = pd.to_numeric(df['year'], errors='coerce')
df = df.astype({
    "deaths" : "float64",
    "population": "float64"
    }    
)
df['cause'] = (
    df['cause']
    .str.replace(r"\(.*?\)","",regex=True)
    .str.replace(r"#","",regex=True)
    .str.strip()
) 
df['crude_rate'] = np.round(df['deaths'] / df['population'] * 1e5,1)
df.head()

,state,year,cause,deaths,population,crude_rate
0,Alabama,2015.0,Certain other intestinal infections,159.0,4858979.0,3.3
1,Alabama,2015.0,Tuberculosis,11.0,4858979.0,0.2
2,Alabama,2015.0,Septicemia,1046.0,4858979.0,21.5
3,Alabama,2015.0,Viral hepatitis,96.0,4858979.0,2.0
4,Alabama,2015.0,Human immunodeficiency virus disease,126.0,4858979.0,2.6


### Convering Data into Facts

To make the dataset usable in a RAG system, each row was converted into a self-contained natural-language statement, referred to as a "fact". This approach enables the embedding model and LLM to process the information in text form, which is more natural for semantic search and generation.

In [3]:
facts = []

for i, row in df.iterrows():
    fact = f"In {row['state']} in {row['year']}, the {row['cause']} mortality rate was {row['crude_rate']} per 100,000."
    facts.append(fact)

print(facts[:5])

['In Alabama in 2015.0, the Certain other intestinal infections mortality rate was 3.3 per 100,000.', 'In Alabama in 2015.0, the Tuberculosis mortality rate was 0.2 per 100,000.', 'In Alabama in 2015.0, the Septicemia mortality rate was 21.5 per 100,000.', 'In Alabama in 2015.0, the Viral hepatitis mortality rate was 2.0 per 100,000.', 'In Alabama in 2015.0, the Human immunodeficiency virus  disease mortality rate was 2.6 per 100,000.']


### Embeddings and Vector Index

Both queries and facts are mapped into a shared semantic vector space using sentence embeddings. This allows the system to retrieve the most relevant facts for a given query based on similarity rather than exact keyword matching.

A FAISS index was built to enable fast near-neighbor search across all fact embeddings. This index is central to the retrieval process in the RAG pipeline.

In [4]:
embedder = SentenceTransformer('all-MiniLM-L6-v2')
fact_embeddings = embedder.encode(facts, convert_to_numpy=True)
print(fact_embeddings.shape)

dimension = fact_embeddings.shape[1]
# set up a search engine that compares vectors by distance
index = faiss.IndexFlatL2(dimension)
# load all the fact embeddings into the search engine so they can be queried later
index.add(fact_embeddings.astype("float32"))

(18805, 384)


### Query Classification Layer

Not all queries are equal. Some are simple lookups (e.g., "What was the heart disease rate in Texas in 2019?"), while others are aggregations (e.g., "What are the top 3 states with the highest heart disease mortality in 2019?").

A rules-based query classification function was implemented to distinguish between these categories. Lookup queries are answered using a small number of retrieved facts to maximize precision. Aggregation queries require a larger set of retrieved facts to provide broader context for the LLM. 

### Retrieval-Augmented Generation (RAG) Pipeline

The complete pipeline works as follows:

1. The user query is classified as either lookup or aggregation.
2. The query is embedded and searched against the FAISS index.
3. The top-k most relevant facts are retrieved.
4. The retrieved facts are passed as context to an LLM running via Ollama.
5. The LLM generates an answer, constrained by the retrieved facts.

This ensures that the model's answers are grounded in actual data whenever possible. See some examples below.

In [5]:
def classify_k(query):
    query = query.lower()
    if any(word in query for word in [
                "top", "highest", "lowest", 
                "average", "most", "least",
                "trend","rank"]):
        return 20
    else:
        return 2

def rag_pipeline(query):
    
    k = classify_k(query)

    query_embeddings = embedder.encode([query], convert_to_numpy=True)
    D, I = index.search(query_embeddings.astype("float32"), k=k)
    retrieved_facts = [facts[i] for i in I[0]]

    prompt = f"""
    You are a helpful assistant. Use only the facts provided to answer the question. 
    If the answer is not in the facts, say "I don't have data on that."

    Facts:
    {retrieved_facts}

    Question:

    {query}

    Answer:
    """
    
    process = subprocess.run(
        ["ollama", "run", "mistral", prompt],
        capture_output=True,
        text=True
    )
    
    print(process.stdout)

In [6]:
rag_pipeline("What was the heart disease mortality rate in Texas in 2019?")

 The total heart disease mortality rate in Texas in 2019 was the sum of the Diseases of heart mortality rate and Other heart diseases mortality rate. So, it would be 159.1 (Diseases of heart mortality rate) + 52.0 (Other heart diseases mortality rate) = 211.1 per 100,000.




In [7]:
rag_pipeline("In which year did Texas have a diabetes mortality rate of 85?")

 I don't have data on that. The provided facts only show the rates for the years 2016 and 2017, neither of which is 85 per 100,000.




In [8]:
rag_pipeline("What was the COVID-19 mortality rate in Texas in 2018?")

 I don't have data on that. The provided facts only mention Influenza and pneumonia mortality rates for Texas in 2018 and 2019, not COVID-19.




In [9]:
rag_pipeline("Rank the top 10 states by heart disease mortality in 2019 in ascending order.")

 To rank the top 10 states by Other heart diseases mortality rate in ascending order for the year 2019, we can arrange the data as follows:

1. Alaska: 42.3 per 100,000 (in Alaska, 2018.0)
2. Washington: 44.8 per 100,000 (in Washington, 2019.0)
3. California: 46.4 per 100,000 (in California, 2018.0)
4. Mississippi: 97.6 per 100,000 (in Mississippi, 2019.0)
5. Massachusetts: 76.2 per 100,000 (in Massachusetts, 2019.0)
6. Maryland: 63.6 per 100,000 (in Maryland, 2019.0)
7. Pennsylvania: Not provided for the year 2019 in the given data set. However, it was 103.0 per 100,000 (in Pennsylvania, 2019.0), and 101.2 per 100,000 (in Pennsylvania, 2020.0).
8. Alabama: Not provided in the given data set. However, we can infer that it must have a higher rate than Pennsylvania's 2019 rate of 103.0 per 100,000 since other states with lower rates (like Massachusetts and Maryland) are ranked above Pennsylvania.
9. Not enough data to determine the remaining rankings for states with missing 2019 data or 

In [10]:
rag_pipeline("What is the trend for Septicemia mortality rate in Arizona from 2015 to 2020?")

 The Septicemia mortality rate in Arizona has a downward trend from 2015 to 2020, as the rate decreased from 6.5 per 100,000 in 2015 to 4.7 per 100,000 in 2019, and then slightly increased to 5.7 per 100,000 in 2020.




### Observations

The system performed well on lookup queries. When only a few facts were retrieved, answers were accurate and concise, demonstrating that this approach works reliably for direct fact-based questions. Aggregation queries performed less consistently. A larger k improved results by giving the LLM more context, but the model sometimes hallucinated rankings or summaries rather than calculating them. For out-of-scope queries such as COVID mortality, the model responds as instructed. 

### Conclusion

This POC demonstrates the feasibility of building a healthcare data Q&A assistant using RAG. The project shows that structured tabular data can be transformed into natural-language facts, that embedding and retrieval provide a scalable foundation for semantic search, and that even a lightweight query classification mechanism improves adaptabiliy. Lookup queries performed strongly, while aggregation queries highlighted the limitations of relying on the LLM without structured computation. Although this system is not production-ready, it validates the overall approach and serves as a strong foundation for future development. 

### Next Steps

The next logical improvements include replacing the rules-based query classifier with an LLM-based approach for greater flexibility, adding structured aggregation logic with tools such as Pandas or SQL to ensure accuracy in summary queries.

Another important enhancement is the addition of validation mechanisms. The system should not only provide an answer, but also present the supporting facts, similarity scores, and links back to the original dataset rows so that users can verify correctness. This transparency is especially critical in the healthcare domain. Further steps include expanding the dataset to test performance at scale with formal evaluation metrics, and wrapping the pipeline in a simple application interface using Streamlit or FastAPI to turn the notebook into a working demo.

In summary , this POC achieved its goal of proving that healthcare data can be queried in natural language using a RAG pipeline. It demonstrates clear strengths, identifies areas for growth, and establishes a solid foundation for moving from prototype to a more advanced application. 